# Modern BERT release

[https://huggingface.co/answerdotai/ModernBERT-base](https://huggingface.co/answerdotai/ModernBERT-base)

In [ ]:
# dec 2024 need to install transformers from main

!pip install git+https://github.com/huggingface/transformers.git


In [1]:
# retry may 2025
!pip install transformers==4.51.0

In [ ]:
!pip install -U torch # says update torch to avoid NAN https://huggingface.co/answerdotai/ModernBERT-base/discussions/59

In [ ]:
# try FA
# === STILL DOESNT WORK ON KAGGLE; HANGS FOR 20 MINS ===
# I WILL USE SDPA ATTENTION INSTEAD
!pip install flash-attn

# use SDPA instead of flash attention

model = AutoModel.from_pretrained(model_id, attn_implementation="sdpa")

In [ ]:
!pip install -U torchvision # says seems is needed for some reason

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForMaskedLM.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, attn_implementation="sdpa")


text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

# To get predictions for the mask:
masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_token_id)

print("Predicted token:", predicted_token)
# Predicted token:  Paris


In [ ]:
# skip may 2025 seems recent torch has it ok
#!pip install triton

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

model.to(device)

text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model(**inputs)

# To get predictions for the mask:
masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_token_id)

print("Predicted token:", predicted_token)
# Predicted token:  Paris

# Redo without FA

`RuntimeError: FlashAttention only supports Ampere GPUs or newer.`



In [ ]:
# dec 2024 need to install transformers from main

!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

# To get predictions for the mask:
masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_token_id)

print("Predicted token:", predicted_token)
# Predicted token:  Paris

# MAY 2025 REDO WITH SDPA

following this github

https://huggingface.co/answerdotai/ModernBERT-base/discussions/56

In [2]:
# import flash_attn_2_cuda as flash_attn_cud
from transformers import AutoTokenizer, AutoModel

# Model name
model_name = "answerdotai/ModernBERT-base"  

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, attn_implementation="sdpa").to('cuda') # <- or "eager"
print(model.config._attn_implementation)

# Test the model
text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model(**inputs)

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

sdpa


W0511 19:41:33.994000 230 torch/_inductor/utils.py:1250] [1/0] Not enough SMs to use max_autotune_gemm mode


In [3]:
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[ 0.3692, -0.4890, -0.9876,  ..., -0.7126, -0.4182, -0.8814],
         [ 0.6172, -0.3927,  0.3895,  ..., -1.1761, -0.3489,  0.6590],
         [ 0.8259, -1.3028, -1.0519,  ..., -0.1143,  2.0917, -0.1481],
         ...,
         [ 1.7963, -0.1180, -1.0425,  ...,  0.6246, -0.8182, -0.2200],
         [ 0.1229,  0.0540,  0.2288,  ...,  0.1279, -0.0781,  0.0955],
         [ 0.2820, -0.0718,  0.0879,  ...,  0.0072, -0.0114, -0.1541]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


# Try to train on token classification

In [4]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [6]:
ner_feature = raw_datasets["train"].features["ner_tags"]


In [7]:
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
label_names = ner_feature.feature.names


In [9]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [10]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, -100]


In [11]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [12]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    3,    0,    0,    7,    0,    0,    0,    0,    7,    0,    0,
            0, -100],
        [-100,    1,    2,    2, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100]])

In [15]:
!pip install seqeval


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=ff325cc287d55699d40549a383725da38673cda4bd62593b0048cdb219446253
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [16]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [17]:
import evaluate

metric = evaluate.load("seqeval")

In [18]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [19]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [20]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

### UPDATE MAY

now that you load for token classification, remember to use same code as found on the github with the SDPA attention

(problem seems to be with torch or flash attention)

In [21]:
model_id = "answerdotai/ModernBERT-base"
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForMaskedLM.from_pretrained(model_id)

from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_id,
    id2label=id2label,
    label2id=label2id,
    attn_implementation="sdpa" # ------------ added May 2025
).to('cuda')

Some weights of ModernBertForTokenClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
model.config.num_labels


9

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="ModernBERT-ft-NER-ConLLU-test",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False, # toggle to true later when it works
    report_to="none",
)

In [26]:
from time import time

In [27]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

start_time = time()
training_metrics = trainer.train()
end_time = time()

W0511 19:46:05.562000 230 torch/_inductor/utils.py:1250] [1/2] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.094584,0.810248,0.870246,0.839176,0.971897
2,0.189900,0.070549,0.867460,0.907607,0.887080,0.980201


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [ ]:
training_metrics

In [ ]:
trainer.push_to_hub(commit_message="Train 3 epochs ModernBERT on ConLLU")

# Test pipeline with this model

training seems to actually work now

In [29]:
from transformers import pipeline

token_classifier = pipeline(
    "token-classification", model=trainer.model, aggregation_strategy="simple", tokenizer=tokenizer
)

Device set to use cuda:0


In [30]:
res = token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

print(res)


[{'entity_group': 'PER', 'score': 0.9150245, 'word': ' Sylvain', 'start': 10, 'end': 18}, {'entity_group': 'ORG', 'score': 0.7513681, 'word': ' Hug', 'start': 32, 'end': 36}, {'entity_group': 'LOC', 'score': 0.5094426, 'word': 'ging', 'start': 36, 'end': 40}, {'entity_group': 'ORG', 'score': 0.6449477, 'word': ' Face', 'start': 40, 'end': 45}, {'entity_group': 'LOC', 'score': 0.9533394, 'word': ' Brooklyn', 'start': 48, 'end': 57}]


In [31]:
avg_token_classifier = pipeline(
    "token-classification", model=trainer.model, aggregation_strategy="average", tokenizer=tokenizer
)

Device set to use cuda:0


In [32]:
res2 = avg_token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

print(res2)

[{'entity_group': 'PER', 'score': 0.6985061, 'word': ' Sylvain', 'start': 10, 'end': 18}, {'entity_group': 'ORG', 'score': 0.51218045, 'word': ' Hugging Face', 'start': 32, 'end': 45}]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:398: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


In [33]:
# try with long names to see if word/token level errors

res3 = avg_token_classifier("My name is Satya Nadellasohn and I work at HuggingFaceCorpo in Brooklyn.")

print(res3)

[{'entity_group': 'PER', 'score': 0.7383739, 'word': ' Satya Nadellasohn', 'start': 10, 'end': 28}, {'entity_group': 'ORG', 'score': 0.58923256, 'word': ' HuggingFaceCorpo', 'start': 42, 'end': 59}]


In [34]:
res4 = token_classifier("My name is Satya Nadellasohn and I work at HuggingFaceCorpo in Brooklyn.")

print(res4)

[{'entity_group': 'PER', 'score': 0.97418386, 'word': ' Satya Nadellasohn', 'start': 10, 'end': 28}, {'entity_group': 'ORG', 'score': 0.72856694, 'word': ' HuggingFaceCorpo', 'start': 42, 'end': 59}, {'entity_group': 'LOC', 'score': 0.9052725, 'word': ' Brooklyn', 'start': 62, 'end': 71}]


### Want to examine probs

in the above, the exact same input leads to 2 different predictions especially weird as the first pipeline misses Brooklyn despite the fact that score seems very large????

In [36]:
import torch

In [37]:
text = "My name is Satya Nadellasohn and I work at HuggingFaceCorpo in Brooklyn."

inputs = tokenizer(text, return_tensors="pt", return_offsets_mapping=True)
offset_mapping = inputs.pop("offset_mapping")

# model predictions
with torch.no_grad():
    outputs = model(**inputs)
    
# logits
logits = outputs.logits[0].detach().cpu().numpy()

probabilities = torch.nn.functional.softmax(outputs.logits, dim=2)[0].detach().cpu().numpy()

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

id2label = model.config.id2label
label_list = [id2label[i] for i in range(len(id2label))]


print("Text:", text)
print("\nDetailed token predictions:")
print("-" * 50)
print(f"{'Token':<15} {'Predicted Label':<15} {'Probabilities'}")
print("-" * 50)

for token, probs in zip(tokens, probabilities):
    # Get the predicted label (highest probability)
    predicted_label_id = np.argmax(probs)
    predicted_label = id2label[predicted_label_id]
    
    probs_formatted = str(probs.round(3))
    
    print(f"{token:<15} {predicted_label:<15} {probs_formatted}")

# raw probability arrays
token_probabilities = probabilities.tolist()
print("\nRaw probability arrays:")
for token, probs in zip(tokens, token_probabilities):
    print(f"{token}: {probs}")

TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <function embedding at 0x7b4fe16760e0>(*(FakeTensor(..., size=(1, s0), dtype=torch.int64), Parameter(FakeTensor(..., device='cuda:0', size=(50368, 768), requires_grad=True)), 50283, None, 2.0, False, False), **{}): got RuntimeError('Unhandled FakeTensor Device Propagation for aten.embedding.default, found two different devices cuda:0, cpu')

from user code:
   File "/usr/local/lib/python3.10/dist-packages/transformers/models/modernbert/modeling_modernbert.py", line 214, in compiled_embeddings
    return self.drop(self.norm(self.tok_embeddings(input_ids)))
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/sparse.py", line 190, in forward
    return F.embedding(

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


# oops model and data not on same device

In [40]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trained_model = trainer.model.to(device) # note im still using the trained model that is in the same notebook, should save it then reload

text = "My name is Satya Nadellasohn and I work at HuggingFaceCorpo in Brooklyn."

inputs = tokenizer(text, return_tensors="pt", return_offsets_mapping=True)
offset_mapping = inputs.pop("offset_mapping")

# tensors to the same device as the model !!
for key in inputs:
    inputs[key] = inputs[key].to(device)

with torch.no_grad():
    outputs = trained_model(**inputs)
    
# logits
logits = outputs.logits[0].detach().cpu().numpy()

probabilities = torch.nn.functional.softmax(outputs.logits, dim=2)[0].detach().cpu().numpy()

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0].cpu())

id2label = trained_model.config.id2label
label_list = [id2label[i] for i in range(len(id2label))]

# === create table ===
print("Text:", text)
print("\nDetailed token predictions:")
print("-" * (20 + sum(len(label) + 8 for label in label_list)))

header = f"{'Token':<20}"
for label in label_list:
    header += f"{label:<8}"
print(header)
print("-" * (20 + sum(len(label) + 8 for label in label_list)))

for token, probs in zip(tokens, probabilities):
    row = f"{token:<20}"
    for i, prob in enumerate(probs):
        row += f"{prob:.4f}  "
    print(row)

print("\nPredicted labels:")
print("-" * 40)
print(f"{'Token':<20} {'Predicted Label':<20}")
print("-" * 40)

for token, probs in zip(tokens, probabilities):
    predicted_label_id = np.argmax(probs)
    predicted_label = id2label[predicted_label_id]
    print(f"{token:<20} {predicted_label:<20}")

# raw probability arrays
print("\nRaw probability arrays:")
token_probabilities = probabilities.tolist()
for token, probs in zip(tokens, token_probabilities):
    print(f"{token}: {probs}")

Text: My name is Satya Nadellasohn and I work at HuggingFaceCorpo in Brooklyn.

Detailed token predictions:
---------------------------------------------------------------------------------------------------------------------------------------
Token               O       B-PER   I-PER   B-ORG   I-ORG   B-LOC   I-LOC   B-MISC  I-MISC  
---------------------------------------------------------------------------------------------------------------------------------------
[CLS]               0.9261  0.0383  0.0002  0.0266  0.0019  0.0053  0.0011  0.0003  0.0001  
My                  0.9787  0.0005  0.0002  0.0098  0.0001  0.0006  0.0002  0.0059  0.0042  
Ġname               0.9883  0.0013  0.0054  0.0018  0.0016  0.0001  0.0004  0.0002  0.0008  
Ġis                 0.9755  0.0020  0.0009  0.0080  0.0092  0.0026  0.0001  0.0008  0.0008  
ĠSat                0.0013  0.9753  0.0207  0.0014  0.0001  0.0007  0.0001  0.0004  0.0000  
ya                  0.0003  0.0001  0.9957  0.0000  0.0018  0.

# Test loading model from HF and pipeline

In [ ]:
del model
del tokenizer

In [ ]:
import gc
import torch


gc.collect()
torch.cuda.empty_cache()

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "benjaminzwhite/ModernBERT-ft-NER-ConLLU-test"

token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)


In [ ]:
token_classifier

In [ ]:
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")


In [ ]:
classifier = pipeline("ner", model=model_checkpoint)

In [ ]:
classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

In [ ]:
ft_model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

In [ ]:
ft_tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [ ]:
sample_text = "My name is Sylvain and I work at Hugging Face in Brooklyn."

In [ ]:
inputs = ft_tokenizer(sample_text, return_tensors="pt")
outputs = ft_model(**inputs)

In [ ]:
outputs

# TODO

Debug, training didn't seem to work